In [1]:
%matplotlib inline
from IPython.core.pylabtools import figsize
import numpy as np
from matplotlib import pyplot as plt
figsize(18, 6)

import pandas as pd
import scipy.stats as stats
import scipy.special as special

import pymc3 as pm
import theano
import theano.tensor as tt

Below, we simulate a process that generated missing data.

It is a totally contrived example:

- We have houses
- Each houses has a songbird
- Each songbird has a certain number of notes
- When a house has a cat the number of notes is lesser (below beta is negative)

In [2]:
N_houses = 100
alpha = 5 # avg nb of notes in houses
beta = -3 # slope for cat influence
k = 0.3 # probability of having a cat
r = 0.2 # probability that we don't know if a house has a cat

With that we can simulate the data

In [3]:
cat = stats.bernoulli.rvs(k,size=N_houses)
cat = cat.astype('float') ## to be able to replace array elements afterwards (having integers would cause an issue)
notes = stats.poisson.rvs(alpha + cat * beta ,size = N_houses)
notes = notes.astype('float')
R_C = stats.bernoulli.rvs(r,size=N_houses) ## house with known and unknown cats
cat_obs = cat
cat_obs[R_C == 1] = -9 ## arbitrary impossible replacement to signal a misssing value

cat = cat.astype('int')
notes = notes.astype('int')

From there we want to go the other way around and pretend we know `notes` while we want to know which are the houses having a `cat`.

If we call Ni the number of notes and Ci the presence of a cat in the house, we have:

Pr(Ni) = (proba of a cat) * (proba of Ni while having a cat) + (proba of no cat) * (proba of Ni while having not a cat)

Pr(Ni) = Pr(Ci = 1) * Pr(Ni | Ci = 1) + Pr(Ci = 0) * Pr(Ni | Ci = 0)




In [13]:
with pm.Model() as cat_1:
    
    k = pm.Beta("k",2,2)
    cat_cond_RC0 = pm.Bernoulli("cat|RC==0",k)
    
    a = pm.Normal("a",0,1)
    b = pm.Normal("b",0,0.5)
    
    # cat NA
    
    essai = pm.math.log(pm.Poisson.dist(pm.math.exp(a+b)).logp(notes[cat_obs == -9]))
    
    def log_sum_exp(a,b):
        c = max(a,b)
        total = c + tt.log(tt.exp(a - c) + tt.exp(b - c))
        return total
    
    
    def custom_density(x,k,a,b):
        ## logsumexp in Pymc takes one vector while Stan takes 2: https://mc-stan.org/docs/2_18/stan-users-guide/summing-out-the-responsibility-parameter.html
        ## I don't understand how to do it with pymc
        
        val = pm.math.logsumexp([tt.log(k) + tt.log(pm.Poisson.dist(pm.math.exp(a+b)).logp(x)),
                                 tt.log(1-k) + tt.log(pm.Poisson.dist(pm.math.exp(a)).logp(x))]
                     )
        return val
        
    
    notes_cond_RC1 = pm.DensityDist("notes|RC==1",custom_density,observed=dict(x=notes[cat_obs == -9],k=k,a=a,b=b))
    
    
    #cat known present / absent
    lambda_ = pm.math.exp(a + b * cat_obs[cat_obs != -9])
    
    notes_cond_RC0 = pm.Poisson("notes|RC==0",lambda_,observed=notes[cat_obs != -9])
    
    trace_cat = pm.sample(1000)

Multiprocess sampling (2 chains in 2 jobs)
CompoundStep
>NUTS: [b, a, k]
>BinaryGibbsMetropolis: [cat|RC==0]
Sampling 2 chains, 0 divergences:   0%|          | 0/3000 [00:00<?, ?draws/s]/opt/anaconda3/envs/data/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/anaconda3/envs/data/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
Sampling 2 chains, 0 divergences:   0%|          | 0/3000 [00:01<?, ?draws/s]
Bad initial energy, check any log probabilities that are inf or -inf, nan or very small:
notes|RC==1   NaN


ParallelSamplingError: Bad initial energy